In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
class StackedAE(nn.Module):
    """
        Description:
            creates a stacked auto encoder, builds off the
            nn.Module class
            
        Functions:
            __init__(self, num_movies, num_nodes_layer1, num_nodes_layer2)
            forward(X)
    """
    def __init__(self, num_movies, num_nodes_layer1, num_nodes_layer2):
        """
            Description:
                initializes the autoencoder
            
            Params:
                num_movies:
                num_nodes_layer1: 
                num_nodes_layer2:
                
            Returns:
                none
        """
        super(StackedAE, self).__init__()
        self.fully_connected1 = nn.Linear(num_movies, num_nodes_layer1)
        self.fully_connected2 = nn.Linear(num_nodes_layer1, num_nodes_layer2)
        self.fully_connected3 = nn.Linear(num_nodes_layer2, num_nodes_layer1)
        self.fully_connected4 = nn.Linear(num_nodes_layer1, num_movies)
        
        self.activation = nn.Sigmoid()
        return
    
    def forward(self, X):
        """
            Description:
                propagates the vector through the network
                
            Params:
                X: the input vector
            
            Returns:
                X: the decoded vector (after transformation)
        """
        X = self.activation(self.fully_connected1(X))
        X = self.activation(self.fully_connected2(X))
        X = self.activation(self.fully_connected3(X))
        X = self.fully_connected4(X)
        return X

In [3]:
movies = pd.read_csv('datasets/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('datasets/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('datasets/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

In [4]:
training_set = pd.read_csv('datasets/ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('datasets/ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

In [5]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))
print(nb_users)
print(nb_movies)

943
1682


In [6]:
def convert(X, M, N):
    """
        converts into list of list for pytorch input
               : { movie 1, movie 2, ... , movie N}
        {
         user 1    rating   rating         rating
         user 2    rating   rating         rating
         ...
         user M    rating   rating         rating
         }
    """
    X_new = []
    
    for user_id in range(1, M+1):
        movies  = X[:,1][X[:,0] == user_id]
        ratings = X[:,2][X[:,0] == user_id]
        
        user_ratings = np.zeros(N)
        user_ratings[movies-1] = ratings
        X_new.append(list(user_ratings))
    
    return X_new

In [7]:
training_set = convert(training_set, nb_users, nb_movies)
test_set = convert(test_set, nb_users, nb_movies)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [8]:
sae = StackedAE(nb_movies, 20, 10)
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

In [9]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
    
    

epoch: 1 loss: 1.7710506888516213
epoch: 2 loss: 1.0965526741299816
epoch: 3 loss: 1.0536392217202608
epoch: 4 loss: 1.0383315494887557
epoch: 5 loss: 1.0310186617410992
epoch: 6 loss: 1.02664469657653
epoch: 7 loss: 1.0237033472805255
epoch: 8 loss: 1.0221581042076942
epoch: 9 loss: 1.0207121525609397
epoch: 10 loss: 1.0198231582757322
epoch: 11 loss: 1.0187973611636554
epoch: 12 loss: 1.0186062798793343
epoch: 13 loss: 1.0179196061120193
epoch: 14 loss: 1.0176833251530533
epoch: 15 loss: 1.017287957948052
epoch: 16 loss: 1.0169221042304004
epoch: 17 loss: 1.01676655694
epoch: 18 loss: 1.0164872578449762
epoch: 19 loss: 1.016467787422786
epoch: 20 loss: 1.0163780495686798
epoch: 21 loss: 1.0160803976389976
epoch: 22 loss: 1.016051250010816
epoch: 23 loss: 1.015731253923503
epoch: 24 loss: 1.015842454222574
epoch: 25 loss: 1.0157935923424255
epoch: 26 loss: 1.0157334747150195
epoch: 27 loss: 1.0155054158282242
epoch: 28 loss: 1.0148915482165644
epoch: 29 loss: 1.0134029690430608
epoch:

In [10]:
nb_movies

1682

In [11]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user])
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data[0]*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

IndexError: The shape of the mask [1682] at index 0 does not match the shape of the indexed tensor [1, 1682] at index 0